In [ ]:
import numpy as np

In [ ]:
def print_matrix(matrix):
    for row in matrix:
        print([round(elem, 4) for elem in row])

In [ ]:
def make_spd_gauss_jordan(A):
    n = A.shape[0]
    if A.shape[1] != n:
        raise ValueError("Matrix must be square.")
    A = 0.5 * (A + A.T)

    for i in range(n):
        if A[i, i] <= 0:
            A[i, i] = abs(A[i, i]) + 1  # Ensure diagonal is positive

        for j in range(n):
            if i != j:
                A[i, j] = (A[i, j] + A[j, i]) / 2  # Force symmetry

    eigvals, _ = np.linalg.eigh(A)
    min_eig = min(eigvals)

    if min_eig <= 0:
        A += np.eye(n) * (abs(min_eig) + 1e-6)  # Shift diagonal to make positive definite

    return A

In [ ]:
def check_positive_defininte(matrix):
    matrix = np.array(matrix)
    if  np.array_equal(matrix, matrix.T):
        eigenvalues = np.linalg.eigvals(matrix)
        print(f"Eigenvalues: {eigenvalues}")
        return np.all(eigenvalues > 0)
    else:
        print("Matrix is not symmetric.")
        return False

In [ ]:
def gauss_jordan_update_with_condition(equations, soll):
    n = len(equations)
    matrix = [equations[i] + [soll[i]] for i in range(n)]  # Augmented matrix [A | b]

    print("Initial Augmented Matrix:")
    print_matrix(matrix)

    for index, eq in enumerate(equations):
        k = eq.copy()  # Copy the row
        k.pop(index)  # Remove the diagonal element

        # Check if the diagonal element is greater than the sum of the rest of the row
        if eq[index] > sum(k):
            print(f"Skipping Gauss-Jordan on row {index} as diagonal element is greater than sum of rest")
            continue

        # Apply Gauss-Jordan elimination if the condition is not met
        print(f"Applying Gauss-Jordan to row {index}")
        pivot = matrix[index][index]

        # Normalize the pivot row (divide by the diagonal element to make it 1)
        for j in range(n + 1):  # Loop over all columns (including the solution part)
            matrix[index][j] /= pivot

        # Eliminate the column above and below the pivot
        for i in range(n):
            if i != index:
                factor = matrix[i][index]
                for j in range(n + 1):  # Loop over all columns (including the solution part)
                    matrix[i][j] -= factor * matrix[index][j]

    print("\nFinal Augmented Matrix:")
    print_matrix(matrix)
    print("\n")

    # Extract the solution vector from the last column
    solution = [matrix[i][n] for i in range(n)]

    # Remove the last column to return the updated matrix without the solution part
    updated_matrix = [row[:n] for row in matrix]

    return solution, updated_matrix

# Lab 1

### Update value of X on every iteration

In [ ]:
def simple_iteration(equations, soll, tolerance=1e-6, max_iterations=20):
    n = len(equations)
    x = [0] * n

    # Construct matrices A* and a*
    A = [[0 if i == j else round(-equations[i][j] / equations[i][i], 4) for j in range(n)] for i in range(n)]
    a = [round(soll[i] / equations[i][i], 4) for i in range(n)]

    print("Matrix A*:")
    print_matrix(A)
    print("Vector a*:", [round(val, 4) for val in a])

    for itr in range(max_iterations):
        new_x = [0] * n
        for i in range(n):
            new_x[i] = round(a[i] + sum(A[i][j] * x[j] for j in range(n)), 4)

        delta = round(max(abs(new_x[i] - x[i]) for i in range(n)), 4)

        print(f"Iteration {itr}: {new_x}, Δ = {delta}")

        if delta < tolerance:
            print("Delta < tolerance!")
            return new_x

        x = new_x

    print("Reached to the maximum number of iterations.")
    return x

### Update value of X on every calculation

In [ ]:
def simple_iteration_with_updates(equations, soll, tolerance=1e-6, max_iterations=20):
    n = len(equations)
    x = [0] * n

    # Construct matrices A* and a*
    A = [[0 if i == j else round(-equations[i][j] / equations[i][i], 4) for j in range(n)] for i in range(n)]
    a = [round(soll[i] / equations[i][i], 4) for i in range(n)]

    print("Matrix A*:")
    print_matrix(A)
    print("Vector a*:", [round(val, 4) for val in a])

    for itr in range(max_iterations):
        new_x = x.copy()
        for i in range(n):
            new_x[i] = round(a[i] + sum(A[i][j] * new_x[j] for j in range(n)), 4)
        delta = round(max(abs(new_x[i] - x[i]) for i in range(n)), 4)

        print(f"Iteration {itr }: {new_x}, Δ = {delta}")

        if delta < tolerance:
            print("Delta < tolerance!")
            return new_x

        x = new_x

    print("Reached to the maximum number of iterations.")
    return x

# Lab 2

In [13]:
def simple_iterations_unobvious(equations, c_matrix, tao, soll, tolerance=1e-6, max_iterations=20):
    a = soll
    c_matrix = np.array(c_matrix)
    equations = np.array(equations)
    if not check_positive_defininte(equations) or not check_positive_defininte(c_matrix):
        print("Matrix is not positive definite.")
        return
    m = 2 * c_matrix - tao * equations
    while not check_positive_defininte(m):
        tao = round(np.random.rand(), 1)
        m = 2 * c_matrix - tao * equations
    tao_c_inv = np.dot(tao, np.linalg.inv(c_matrix))
    ab = np.dot(tao_c_inv, equations)
    cd = np.dot(tao_c_inv, a)
    x = [0] * (len(equations))
    I = np.eye(len(equations))
    for itr in range(1, max_iterations+1):
        new_x = x.copy()
        new_x = np.round(np.dot(I-ab, x) + cd, 4)
        delta = round(max(abs(new_x - x)), 4)
        if (itr+1) % 1 == 0 or itr+1 == 1:
            print(f"Iteration {itr}: {new_x}, delta = {delta}")
        if delta < tolerance:
            print("Delta < tolerance!")
            # print(f"Iteration {itr + 1}: {new_x}, delta = {delta}")
            return new_x
        x = new_x

# Lab 3

### Modifikacvac

In [ ]:
def simple_iterations_mod(equations, c_matrix, tao, soll, tolerance=1e-4, max_iterations=20):
    a = soll
    n = len(equations)
    d = np.diag(np.diag(equations))
    equations = np.array(equations)
    if not check_positive_defininte(equations):
        print("Matrix is not positive definite.")
        return
    d_inv = np.linalg.inv(d)
    ab = np.dot(d_inv, equations)
    cd = np.dot(d_inv, a)
    x = [0] * (len(equations))
    I = np.eye(len(equations))
    for itr in range(1, max_iterations+1):
        new_x = x.copy()
        new_x = np.round(np.dot(I-ab, new_x) + cd, 4)
        delta = round(max(abs(new_x - x)), 4)
        if (itr+1) % 1 == 0 or itr+1 == 1:
            print(f"Iteration {itr}: {new_x}, delta = {delta}")
        if delta < tolerance:
            print("Delta < tolerance!")
            print(f"Iteration {itr}: {new_x}, delta = {delta}")
            return new_x
        x = new_x

### Seidel

In [ ]:
def simple_iterations_seidel(equations, c_matrix, tao, soll, tolerance=1e-4, max_iterations=20):
    a = soll
    n = len(equations)
    d = np.diag(np.diag(equations))
    equations = np.array(equations)
    if not check_positive_defininte(equations):
        print("Matrix is not positive definite.")
        return
    V = np.tril(equations, k=-1)
    dV_inv = np.linalg.inv(d+V)
    ab = np.dot(dV_inv, equations)
    cd = np.dot(dV_inv, a)
    x = [0] * (len(equations))
    I = np.eye(len(equations))
    for itr in range(1, max_iterations+1):
        new_x = x.copy()
        new_x = np.round(np.dot(I-ab, x) + cd, 4)
        delta = round(max(abs(new_x - x)), 4)
        if (itr+1) % 1 == 0 or itr+1 == 1:
            print(f"Iteration {itr}: {new_x}, delta = {delta}")
        if delta < tolerance:
            print("Delta < tolerance!")
            print(f"Iteration {itr }: {new_x}, delta = {delta}")
            return new_x
        x = new_x

In [16]:
equations = [[4, 1, -1],
             [1, 2, 0],
             [-1, 0, 3]]

c_matrix = [[0.5, 0.1, 0.1],
            [0.1, 0.5, 0],
            [0.1, 0, 0.5]]

soll = [7, 0, -2]

tao = round(np.random.rand(), 1)
tao = 0.1
# soll, equations = gauss_jordan_update_with_condition(equations, soll)
print("====================== Lab 1 ===============================================================")
a = simple_iteration(equations, soll)
b = simple_iteration_with_updates(equations, soll)
print("====================== Lab 2 ===============================================================")
a = simple_iterations_unobvious(equations, c_matrix, tao, soll, tolerance=1e-2)
print("====================== Lab 3 ===============================================================")
tao = 1
print("-------------- Modifikacvac --------------")
a = simple_iterations_mod(equations, c_matrix, tao, soll)
print("-------------- Zeidel --------------")
a = simple_iterations_seidel(equations, c_matrix, tao, soll)


====================== Lab 1 ===============================================================
Matrix A*:
[0, -0.25, 0.25]
[-0.5, 0, 0.0]
[0.3333, 0.0, 0]
Vector a*: [1.75, 0.0, -0.6667]
Iteration 0: [1.75, 0.0, -0.6667], Δ = 1.75
Iteration 1: [1.5833, -0.875, -0.0834], Δ = 0.875
Iteration 2: [1.9479, -0.7916, -0.139], Δ = 0.3646
Iteration 3: [1.9131, -0.9739, -0.0175], Δ = 0.1823
Iteration 4: [1.9891, -0.9566, -0.0291], Δ = 0.076
Iteration 5: [1.9819, -0.9946, -0.0037], Δ = 0.038
Iteration 6: [1.9977, -0.9909, -0.0061], Δ = 0.0158
Iteration 7: [1.9962, -0.9989, -0.0009], Δ = 0.008
Iteration 8: [1.9995, -0.9981, -0.0014], Δ = 0.0033
Iteration 9: [1.9992, -0.9998, -0.0003], Δ = 0.0017
Iteration 10: [1.9999, -0.9996, -0.0004], Δ = 0.0007
Iteration 11: [1.9998, -1.0, -0.0001], Δ = 0.0004
Iteration 12: [2.0, -0.9999, -0.0002], Δ = 0.0002
Iteration 13: [1.9999, -1.0, -0.0001], Δ = 0.0001
Iteration 14: [2.0, -1.0, -0.0001], Δ = 0.0001
Iteration 15: [2.0, -1.0, -0.0001], Δ = 0.0
Delta < toleran

In [89]:
def simple_iterations_min_square(equations, soll, tolerance=1e-6, max_iterations=25, x=[0, 0, 0]):
    equations = np.array(equations)
    AT = np.dot(np.transpose(equations), equations)
    d = np.diag(np.diag(AT))
    V = np.tril(AT, k=-1)
    dV_inv = np.linalg.inv(d+V)
    I = np.eye(len(equations))
    cd = np.dot(dV_inv, np.dot(np.transpose(equations), soll))
    print(cd)
    for itr in range(1, max_iterations+1):
        new_x = x.copy()
        new_x = np.round(np.dot(I-np.dot(dV_inv, AT), x)+ cd, 4)
        delta = round(max(abs(new_x - x)), 4)
        if (itr) % 5 == 0 or itr == 1:
            print(f"Iteration {itr}: {new_x}, delta = {delta}")
        if delta < tolerance:
            print("Delta < tolerance!")
            print(f"Iteration {itr }: {new_x}, delta = {delta}")
            return new_x
        x = new_x
    return x

In [91]:
equations = [[0, 1, 3],
             [5, -1, 1],
             [2, -2, 2]]

soll = [0, 0, 1]
simple_iterations_min_square(equations, soll, x=[-10, -10, 10])

[ 0.06896552 -0.22988506  0.06568144]
Iteration 1: [-6.1379 -6.2069  3.202 ], delta = 6.798
Iteration 5: [-0.4858 -0.8999  0.3266], delta = 0.4012
Iteration 10: [-0.1384 -0.4848  0.1626], delta = 0.0149
Iteration 15: [-0.1255 -0.4693  0.1565], delta = 0.0006
Iteration 20: [-0.125  -0.4688  0.1563], delta = 0.0
Delta < tolerance!
Iteration 20: [-0.125  -0.4688  0.1563], delta = 0.0


array([-0.125 , -0.4688,  0.1563])

In [92]:
equations = [[2.7, 3.3, 1.3],
             [3.5, -1.7, 2.8],
             [4.1, 5.8, -1.7]]
soll = [2.1, 1.7, 0.8]
simple_iterations_min_square(equations, soll, tolerance=1e-6, max_iterations=100)


[ 0.40990371 -0.04809838  0.24431193]
Iteration 1: [ 0.4099 -0.0481  0.2443], delta = 0.4099
Iteration 5: [0.2771 0.115  0.4477], delta = 0.043
Iteration 10: [0.1637 0.2144 0.5883], delta = 0.0204
Iteration 15: [0.1099 0.2617 0.6551], delta = 0.0097
Iteration 20: [0.0843 0.2842 0.6869], delta = 0.0046
Iteration 25: [0.0721 0.2948 0.702 ], delta = 0.0022
Iteration 30: [0.0663 0.2999 0.7092], delta = 0.0011
Iteration 35: [0.0635 0.3023 0.7126], delta = 0.0005
Iteration 40: [0.0623 0.3034 0.7142], delta = 0.0003
Iteration 45: [0.0617 0.304  0.7149], delta = 0.0001
Iteration 50: [0.0614 0.3043 0.7153], delta = 0.0001
Delta < tolerance!
Iteration 52: [0.0613 0.3043 0.7154], delta = 0.0


array([0.0613, 0.3043, 0.7154])

In [94]:
equations = [[1.7, 2.8, 1.9],
             [2.1, 3.4, 1.8],
             [4.2, -1.7, 1.3]]
soll = [0.7, 1.1, 2.8]
simple_iterations_min_square(equations, soll, tolerance=1e-6, max_iterations=1000)

[ 0.61186848 -0.08849233  0.01601688]
Iteration 1: [ 0.6119 -0.0885  0.016 ], delta = 0.6119
Iteration 5: [ 0.6299 -0.0923 -0.0061], delta = 0.0061
Iteration 10: [ 0.6429 -0.0828 -0.0354], delta = 0.0057
Iteration 15: [ 0.655  -0.0739 -0.0627], delta = 0.0053
Iteration 20: [ 0.6663 -0.0657 -0.088 ], delta = 0.0049
Iteration 25: [ 0.6768 -0.058  -0.1118], delta = 0.0046
Iteration 30: [ 0.6867 -0.0508 -0.134 ], delta = 0.0043
Iteration 35: [ 0.6958 -0.0441 -0.1546], delta = 0.004
Iteration 40: [ 0.7044 -0.0378 -0.1738], delta = 0.0037
Iteration 45: [ 0.7123 -0.032  -0.1917], delta = 0.0035
Iteration 50: [ 0.7197 -0.0266 -0.2083], delta = 0.0032
Iteration 55: [ 0.7266 -0.0215 -0.2239], delta = 0.003
Iteration 60: [ 0.733  -0.0168 -0.2384], delta = 0.0028
Iteration 65: [ 0.7391 -0.0124 -0.2519], delta = 0.0026
Iteration 70: [ 0.7446 -0.0084 -0.2644], delta = 0.0024
Iteration 75: [ 0.7498 -0.0046 -0.2761], delta = 0.0023
Iteration 80: [ 0.7546 -0.001  -0.2869], delta = 0.0021
Iteration 85: 

array([ 0.8187,  0.0459, -0.4312])

In [ ]:
equations = [[3.1, 2.8, 1.9],
             [1.9, 3.1, 2.1],
             [7.5, 3.8, 4.8]]
soll = [0.2, 2.1, 5.6]
soll, equations = gauss_jordan_update_with_condition(equations, soll)

print("Iterate and update x after every iteration")
solution = simple_iteration(equations, soll, tolerance=0.001)
print("\nFinal solution:", [round(val, 4) for val in solution])
print("\n------------------------------------\n")
print("Iterate and update x after every calculation")
solution = simple_iteration_with_updates(equations, soll, tolerance=0.001)
print("\nFinal solution:", [round(val, 4) for val in solution])
print("\n------------------------------------\n")
c_matrix = [[0.5, 0.1, 0.1],
            [0.1, 0.5, 0],
            [0.1, 0, 0.5]]
tao = round(np.random.rand(), 1)
a = simple_iterations_unobvious(equations, c_matrix, tao, soll)

In [ ]:
equations = [[9.1, 5.6, 7.8],
             [3.8, 5.1, 2.8],
             [4.1, 5.7, 1.2]]
soll = [9.8, 6.7, 5.8]
soll, equations = gauss_jordan_update_with_condition(equations, soll)

print("Iterate and update x after every iteration")
solution = simple_iteration(equations, soll, tolerance=0.001)
print("\nFinal solution:", [round(val, 4) for val in solution])
print("\n------------------------------------\n")
print("Iterate and update x after every calculation")
solution = simple_iteration_with_updates(equations, soll, tolerance=0.001)
print("\nFinal solution:", [round(val, 4) for val in solution])
print("\n------------------------------------\n")
c_matrix = [[0.5, 0.1, 0.1],
            [0.1, 0.5, 0],
            [0.1, 0, 0.5]]
tao = round(np.random.rand(), 1)
a = simple_iterations_unobvious(equations, c_matrix, tao, soll)

In [ ]:
equations = [[5.4, -2.3, 3.4],
             [4.2, 1.7, -2.3],
             [3.4, 2.4, 7.4]]
soll = [-3.5, 2.7, 1.9]
soll, equations = gauss_jordan_update_with_condition(equations, soll)

print("Iterate and update x after every iteration")
solution = simple_iteration(equations, soll, tolerance=0.001)
print("\nFinal solution:", [round(val, 4) for val in solution])
print("\n------------------------------------\n")
print("Iterate and update x after every calculation")
solution = simple_iteration_with_updates(equations, soll, tolerance=0.001)
print("\nFinal solution:", [round(val, 4) for val in solution])
print("\n------------------------------------\n")
c_matrix = [[0.5, 0.1, 0.1],
            [0.1, 0.5, 0],
            [0.1, 0, 0.5]]
tao = round(np.random.rand(), 1)
a = simple_iterations_unobvious(equations, c_matrix, tao, soll)

In [ ]:
equations = [[2.7, 0.9, -1.5],
             [4.5, -2.8, 6.7],
             [5.1, 3.7, -1.4]]
soll = [3.5, 2.6, -0.14]
soll, equations = gauss_jordan_update_with_condition(equations, soll)

print("Iterate and update x after every iteration")
solution = simple_iteration(equations, soll, tolerance=0.001)
print("\nFinal solution:", [round(val, 4) for val in solution])
print("\n------------------------------------\n")
print("Iterate and update x after every calculation")
solution = simple_iteration_with_updates(equations, soll, tolerance=0.001)
print("\nFinal solution:", [round(val, 4) for val in solution])
print("\n------------------------------------\n")
c_matrix = [[0.5, 0.1, 0.1],
            [0.1, 0.5, 0],
            [0.1, 0, 0.5]]
tao = round(np.random.rand(), 1)
a = simple_iterations_unobvious(equations, c_matrix, tao, soll)

In [ ]:
equations = [[4, 1, -1],
             [1, 2, 0],
             [-1, 0, 3]]

c_matrix = [[0.5, 0.1, 0.1],
            [0.1, 0.5, 0],
            [0.1, 0, 0.5]]

tao = round(np.random.rand(), 1)
tao = 0.1
soll = [7, 0, -2]
a = simple_iterations_unobvious(equations, c_matrix, tao, soll)

In [ ]:
equations = [[1.7, 2.8, 1.9],
             [2.1, 3.4, 1.8],
             [4.2, -1.7, 1.3]]
soll = [0.7, 1.1, 2.8]
soll, equations = gauss_jordan_update_with_condition(equations, soll)

print("Iterate and update x after every iteration")
solution = simple_iteration(equations, soll, tolerance=0.001)
print("\nFinal solution:", [round(val, 4) for val in solution])
print("\n------------------------------------\n")
print("Iterate and update x after every calculation")
solution = simple_iteration_with_updates(equations, soll, tolerance=0.001)
print("\nFinal solution:", [round(val, 4) for val in solution])
print("\n------------------------------------\n")

tao = round(np.random.rand(), 1)
a = simple_iterations_unobvious(equations, c_matrix, tao, soll)

In [ ]:
equations = [[3.6, 1.8, -4.7],
             [2.7, -3.6, 1.9],
             [1.5, 4.5, 3.3]]
soll = [3.8, 0.4, -1.6]
# soll, equations = gauss_jordan_update_with_condition(equations, soll)

print("Iterate and update x after every iteration")
solution = simple_iteration(equations, soll, tolerance=0.001)
print("\nFinal solution:", [round(val, 4) for val in solution])
print("\n------------------------------------\n")
print("Iterate and update x after every calculation")
solution = simple_iteration_with_updates(equations, soll, tolerance=0.001)
print("\nFinal solution:", [round(val, 4) for val in solution])
print("\n------------------------------------\n")

tao = round(np.random.rand(), 1)
a = simple_iterations_unobvious(equations, c_matrix, tao, soll)

In [ ]:
equations = [[3.8, 4.1, -2.3],
             [-2.1, 3.9, -5.8],
             [1.8, 1.1, -2.1]]
soll = [4.8, 3.3, 5.8]
# soll, equations = gauss_jordan_update_with_condition(equations, soll)

print("Iterate and update x after every iteration")
solution = simple_iteration(equations, soll, tolerance=0.001)
print("\nFinal solution:", [round(val, 4) for val in solution])
print("\n------------------------------------\n")
print("Iterate and update x after every calculation")
solution = simple_iteration_with_updates(equations, soll, tolerance=0.001)
print("\nFinal solution:", [round(val, 4) for val in solution])
print("\n------------------------------------\n")

tao = round(np.random.rand(), 1)
a = simple_iterations_unobvious(equations, c_matrix, tao, soll)